# Домашнее задание к лекции "Базовые понятия статистики"

In [312]:
import pandas as pd
import numpy as np

## Задание 1. Базовое изучение

In [319]:
# https://raw.githubusercontent.com/obulygin/pyda_homeworks/master/statistics_basics/horse_data.names
headers = [
    'surgery',
    'age',
    'hospital_number',
    'rectal temperature',
    'pulse',
    'respiratory rate',
    'temperature of extremities',
    'peripheral pulse',
    'mucous membranes',
    'capillary refill time',
    'pain',
    'peristalsis',
    'abdominal distension',
    'nasogastric tube',
    'nasogastric reflux',
    'nasogastric reflux PH',
    'rectal examination',
    'abdomen',
    'packed cell volume',
    'total protein',
    'abdominocentesis appearance',
    'abdomcentesis total protein',
    'outcome',
    'surgical lesion',
    'type of lesion 1',
    'type of lesion 2',
    'type of lesion 3',
    'cp_data'
]

all_df = pd.read_csv('horse_data.csv', names=headers, na_values="?")

# df = all_df[['surgery', 'age', 'hospital_number', 'pulse', 'rectal temperature', 'respiratory rate', 'pain', 'outcome']]

df = all_df.filter(items=['surgery', 'age', 'hospital_number', 'pulse', 'rectal temperature', 'respiratory rate', 'pain', 'outcome'])

Столбцы для исследования:
* surgery (категориальный) - была операция или нет, 1 - да / 2 - нет
* age (категориальный) - возраст, 1 - взрослая / 2 - молодая (но в наборе данных значения 1 и 9, видимо ошибка заполнения файла)
* hospital_number (категориальные) - как понял из описания, это номер карточки лошади, могут быть дубли, так как одну и ту же
* pulse (числовой) - пульс
* rectal temperature (числовой) - ректальная температура
* respiratory rate (числовой) - частота дыхания, нормальные значения от 8 до 10
* pain (категориальные) - от 1 до 5, боль, субъективная оценка ветеринара 
* outcome (категориальные) - от 1 до 3-х, результат осмотра, 1 - выжил, 2 - умер, 3 - усыплен

In [227]:
def get_outliers(column_name):
    q1 = df[column_name].quantile(0.25)
    q3 = df[column_name].quantile(0.75)

    return q1 - 1.5 * (q3 - q1), q3 + 1.5 * (q3 - q1)

def print_num_desc(column_name, title):
    print(f"-----------------------------------")
    print(f"num column = {column_name}, {title}")

    series = df[column_name]
    print(f"% пустых значений: {series.isna().mean() :.1%} %")
    print(f"минимум: {series.min()}")
    print(f"максимум: {series.max()}")
    print(f"разброс: {series.max() - series.min()}")
    print(f"средне арифметическое: {series.mean()}")
    print(f"медиана: {series.median()}")
    print(f"медиана 25%: {series.quantile(0.25)}")
    print(f"медиана 75%: {series.quantile(0.75)}")
    print(f"мода: {series.mode()[0]}")
    print(f"стандартное отклонение: {series.std()}")
    print(f"дисперсия: {series.var()}")
    
def print_cat_desc(column_name, title):
    print(f"-----------------------------------")
    print(f"cat column = {column_name}, {title}")

    series = df[column_name]
    
    print(f"% пустых значений: {series.isna().mean() :.1%}")
    # print(f"уникальные значения {series.unique()}")
    print(f"мода: {series.mode()[0]}")

In [228]:
print_cat_desc('surgery', 'Операция')
print_cat_desc('age', 'Возраст')
print_cat_desc('hospital_number', 'Номер')
print_num_desc('pulse', 'Пульс')
print_num_desc('rectal temperature', 'Температура')
print_num_desc('respiratory rate', 'Частота дыхания')
print_cat_desc('pain', 'Боль')
print_cat_desc('outcome', 'Результат')

-----------------------------------
cat column = surgery, Операция
% пустых значений: 0.3%
мода: 1.0
-----------------------------------
cat column = age, Возраст
% пустых значений: 0.0%
мода: 1
-----------------------------------
cat column = hospital_number, Номер
% пустых значений: 0.0%
мода: 527544
-----------------------------------
num column = pulse, Пульс
% пустых значений: 8.0% %
минимум: 30.0
максимум: 184.0
разброс: 154.0
средне арифметическое: 71.91304347826087
медиана: 64.0
медиана 25%: 48.0
медиана 75%: 88.0
мода: 48.0
стандартное отклонение: 28.630556660735003
дисперсия: 819.7087747035575
-----------------------------------
num column = rectal temperature, Температура
% пустых значений: 20.0% %
минимум: 35.4
максимум: 40.8
разброс: 5.399999999999999
средне арифметическое: 38.16791666666669
медиана: 38.2
медиана 25%: 37.8
медиана 75%: 38.5
мода: 38.0
стандартное отклонение: 0.7322886641121578
дисперсия: 0.5362466875871686
-----------------------------------
num column = r

## Задание 2. Работа с выбросами

### Пульс

In [229]:
low, high = get_outliers('pulse')

df[df.pulse > high].head(10)

,surgery,age,hospital_number,pulse,rectal temperature,respiratory rate,pain,outcome
3,1.0,9,5290409,164.0,39.1,84.0,2.0,2.0
41,2.0,9,5288249,150.0,39.0,72.0,NaN,1.0
55,1.0,9,5282839,160.0,38.6,20.0,3.0,2.0
255,1.0,9,5294539,184.0,38.8,84.0,4.0,2.0
275,1.0,9,5297159,150.0,38.8,50.0,5.0,2.0


Выброс по пульсу наблюдается у молодых лошадей, большая часть их них умерла. Это явно не ошибка сбора данных.

Оставляем данные

### Ректальная температура

In [230]:
low, high = get_outliers('rectal temperature')

df[df['rectal temperature'] > high].head(100)

,surgery,age,hospital_number,pulse,rectal temperature,respiratory rate,pain,outcome
20,1.0,1,530157,72.0,39.9,60.0,5.0,1.0
54,2.0,1,529461,114.0,40.3,36.0,2.0,3.0
75,1.0,9,534092,100.0,39.7,NaN,2.0,3.0
91,2.0,1,529461,114.0,40.3,36.0,2.0,2.0
99,2.0,1,530002,108.0,39.6,51.0,2.0,1.0
259,1.0,1,527734,72.0,40.8,42.0,2.0,2.0
281,2.0,1,530624,78.0,40.0,NaN,2.0,2.0


Выброс так же обусловлен состоянием лошадей, а не замерами. Выжило только две лошади, так же у все лошадей из выброса высокая степень боли.

Оставляем данные

### Частота дыхания

In [231]:
low, high = get_outliers('respiratory rate')

df[df['respiratory rate'] > high].head(100)

,surgery,age,hospital_number,pulse,rectal temperature,respiratory rate,pain,outcome
3,1.0,9,5290409,164.0,39.1,84.0,2.0,2.0
39,1.0,9,5277409,146.0,39.2,96.0,NaN,2.0
41,2.0,9,5288249,150.0,39.0,72.0,NaN,1.0
82,1.0,9,5290759,100.0,38.1,80.0,3.0,1.0
84,1.0,1,529849,60.0,37.8,80.0,2.0,1.0
103,1.0,9,5292489,140.0,38.0,68.0,3.0,1.0
106,1.0,1,528668,52.0,38.3,96.0,NaN,1.0
120,1.0,1,528682,54.0,39.4,66.0,2.0,1.0
125,1.0,1,528461,42.0,38.0,68.0,3.0,1.0
186,1.0,1,5278331,64.0,39.3,90.0,NaN,1.0


В данном случае кол-во записей в выбросе больше. По столбцу pain можно сделать предположить, что повышенная частота дыхания связана с испытываемой болью.

Оставляем данные

## Задание 3. Работа с пропусками

In [261]:
for col in df.columns:
    print(f"{col} - {df[col].isna().mean() : .1%}")

surgery -  0.3%
age -  0.0%
hospital_number -  0.0%
pulse -  8.0%
rectal temperature -  20.0%
respiratory rate -  19.3%
pain -  18.3%
outcome -  0.3%


### Была операция или нет, степень боли, результат

Пропуски в surgery и outcome не значительны, заполняем модами

In [328]:
df['surgery'].fillna(df['surgery'].mode()[0], inplace=True)
df['pain'].fillna(df['pain'].mode()[0], inplace=True)
df['outcome'].fillna(df['outcome'].mode()[0], inplace=True)

print(f"surgery isna = {df['surgery'].isna().mean()}")
print(f"pain isna = {df['pain'].isna().mean()}")
print(f"outcome isna = {df['outcome'].isna().mean()}")

surgery isna = 0.0
pain isna = 0.0
outcome isna = 0.0


### Температура

Заполняю пропуски медианой, так как не могу предложить гипотезу от чего может зависить температура

In [322]:
df['rectal temperature'].fillna(df['rectal temperature'].median(), inplace=True)

print(f"rectal temperature isna = {df['rectal temperature'].isna().mean()}")

rectal temperature isna = 0.0


### Пульс и частота дыхания

Выдвигаю гипотезу, что пульс м частота дыхания связаны между собой, проверяем

In [323]:
df.groupby(['pulse'])['respiratory rate'].median()

pulse
30.0     12.0
36.0     16.0
38.0     20.0
40.0     16.0
42.0     25.0
44.0     14.0
45.0     14.0
46.0     18.4
48.0     19.6
49.0     19.6
50.0     32.0
52.0     20.4
54.0     24.0
56.0     22.4
57.5     23.0
60.0     24.0
64.0     36.0
65.0     24.0
66.0     20.0
68.0     20.0
70.0     19.0
72.0     30.0
75.0     36.0
76.0     29.2
78.0     30.0
80.0     34.0
82.0     12.0
84.0     36.0
86.0     24.0
88.0     33.6
90.0     36.0
92.0     28.0
96.0     33.0
98.0     35.0
100.0    40.0
104.0    40.0
108.0    51.0
110.0    35.0
112.0    16.0
114.0    36.0
120.0    48.0
124.0    62.0
128.0    36.0
129.0    48.0
130.0    38.0
132.0    52.8
136.0    48.0
140.0    46.0
146.0    96.0
150.0    61.0
160.0    20.0
164.0    84.0
184.0    84.0
Name: respiratory rate, dtype: float64

Да, есть зависимости частота дыхания ~ пульс. Используя это, можно заполнить часть пропусков в этих столбцах. Вычисляем множитель (зависимость явно сложнее, но для обучения пойдет)

In [329]:
rate = df.apply(lambda x: round(x['pulse'] / x['respiratory rate'], 1), axis=1).median()

print(f"rate = {rate}")

df['respiratory rate'] = df.apply(
    lambda r: r['pulse'] / rate if np.isnan(r['respiratory rate']) else r['respiratory rate'],
    axis=1
)

df['pulse'] = df.apply(
    lambda r: r['respiratory rate'] * rate if np.isnan(r['pulse']) else r['pulse'],
    axis=1
)

print(f"respiratory rate isna = {df['respiratory rate'].isna().mean()}")
print(f"pulse = {df['pulse'].isna().mean()}")

df.head(20)

rate = 2.5
respiratory rate isna = 0.06
pulse = 0.06


,surgery,age,hospital_number,pulse,rectal temperature,respiratory rate,pain,outcome
0,2.0,1,530101,66.0,38.5,28.0,5.0,2.0
1,1.0,1,534817,88.0,39.2,20.0,3.0,3.0
2,2.0,1,530334,40.0,38.3,24.0,3.0,1.0
3,1.0,9,5290409,164.0,39.1,84.0,2.0,2.0
4,2.0,1,530255,104.0,37.3,35.0,3.0,2.0
5,2.0,1,528355,NaN,38.2,NaN,2.0,1.0
6,1.0,1,526802,48.0,37.9,16.0,3.0,1.0
7,1.0,1,529607,60.0,38.2,24.0,3.0,2.0
8,2.0,1,530051,80.0,38.2,36.0,4.0,3.0
9,2.0,9,5299629,90.0,38.3,36.0,5.0,1.0


Остается 6% пропусков для пульса и частоты дыхания, можно их удалить или заменить медианой (?!)